In [ ]:
from datetime import datetime
import os

from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tables as tb

In [ ]:
ipath = os.path.expanduser("~/Dropbox_LSST/Dome_Seeing_Monitor/DSM_Data/20200122/100FPS")
date_str = "20200122_195044"

In [ ]:
ifname_psd = os.path.join(ipath, f"smm_psd_{date_str}.h5")

# Get data keys from file
psd_h5 = tb.open_file(ifname_psd)
glist = psd_h5.root._f_list_nodes(classname="Group")
keys = [k._v_name for k in glist]
#print(keys)

In [ ]:
x_data = []
y_data = []
frequencies = None
datetimes = []
timestamps = []
#print(keys)
for i, key in enumerate(keys):
    pd_h5 = pd.read_hdf(ifname_psd, key=key)
    x_data.append(pd_h5.X.values)
    y_data.append(pd_h5.Y.values)
    datetimes.append(datetime.strptime(key.replace('DT_', ''), '%Y%m%d_%H%M%S'))
    timestamps.append(datetimes[-1].timestamp())
    if i == 0:
        frequencies = pd_h5.Frequencies.values

In [ ]:
x = np.stack(x_data)
y = np.stack(y_data)

In [ ]:
extent = (frequencies[0], frequencies[-1], timestamps[-1], timestamps[0])
xlabel = r'$\nu$ [1/s]'
plt.style.use('./plot_style.mplstyle')
fig = plt.figure(figsize=(14, 7))
fig.tight_layout()
ax1 = fig.add_subplot(121)
logNormX = LogNorm(vmin=np.min(x), vmax=np.max(x))
pos1 = ax1.imshow(x, aspect='auto', extent=extent, norm=logNormX)
date_fmt = [datetime.strftime(datetime.fromtimestamp(d), '%Y-%m-%d %H:%M:%S') for d in ax1.get_yticks()]
ax1.set_title('PSD X')
ax1.set_yticklabels(date_fmt)
ax1.set_xlabel(xlabel)
fig.colorbar(pos1, orientation='horizontal')

ax2 = fig.add_subplot(122)
logNormY = LogNorm(vmin=np.min(y), vmax=np.max(y))
pos2 = ax2.imshow(y, aspect='auto', extent=extent, norm=logNormY)
ax2.set_title('PSD Y')
ax2.set_yticks([])
plt.setp(ax2.get_yticklabels(), visible=False)
ax2.set_xlabel(xlabel)
fig.colorbar(pos2, orientation='horizontal')

fig.subplots_adjust(wspace=0.01)